# Study A
This notebook shows the full workflow for building models, simulating growth and obtaining SCFA predictions from data collected by the _ex vivo_ study conducted by the Gibbons Lab in 2019

In [ ]:
import pandas as pd
import micom
import micom.measures
from plotnine import * 
import scipy

%matplotlib inline

## SCFA Flux
Here we will gather the net SCFA production across the culturing period. The "Measured" columns contain the measured flux over the course of the experiment (mM/h, [T1]-[T0]/h), while the "Baseline" columns contain the T0 concentrations (mM)

In [ ]:
scfa=pd.read_csv('../data/raw_data/studyA_original_scfas.csv', index_col = 0)

scfa['donor'] = scfa.index.str.split('_').str[0]
scfa['treatment'] = scfa.index.str.split('_').str[1]

scfa = scfa.groupby(['donor','treatment']).mean().reset_index()

scfa['sample_id'] = scfa['donor']+'_'+scfa['treatment']
scfa.set_index('sample_id', inplace = True)

scfa

## Taxonomy Table
Next, we will read in abundance data for all samples. We will convert this into a taxnomy table to use in MICOM

In [ ]:
abundance = pd.read_csv('../data/raw_data/studyA_original_abundance.csv', index_col = 0)
abundance = abundance.rename(columns = 
                           {'d':'domain',
                            'p':'phylum', 
                            'c':'class',
                            'o':'order',
                            'f':'family',
                            'g':'genus',
                            's':'species',
                            'reads':'abundance'})
abundance = abundance.groupby(['sample_id','species']).sum().reset_index()
table = pd.pivot_table(abundance, 
               index = 'sample_id',
               columns = 'species',
               values = 'abundance').fillna(0.0) # build abundance matrix
abundance['species'] = abundance['species'].str.split(' ').str[1].str.lower()
abundance['id'] = abundance['species']
abundance = abundance[abundance['sample_id'].str.contains('|'.join(['control','inulin']))]

abundance

## Build Models
Now, we'll build our models, with cutoff of 0.001

In [ ]:
manifest = micom.workflows.build(abundance,
                               out_folder = '../models/studyA/',
                               model_db = '../agora103_species.qza',
                               cutoff = 0.001, 
                               threads = 10)

## Load Medium
Load in the study-specific carbon-stripped European Diet with dilution-adjusted residual fiber, and construct the intervention diets by augmenting with inulin

In [ ]:
medium = pd.read_csv('../media/studyAmedium.csv', index_col = 0)

controlMedium = medium

inulinMedium = pd.concat([medium,
                           pd.DataFrame({'reaction':['EX_inulin_m'],
                                         'metabolite':['inulin_m'],
                                         'global_id':['EX_inulin(e)'],
                                         'flux':[14]
                                        },index = ['EX_inulin_m'])])
medium

# Grow Models
Now we'll grow the samples using the respective media we constructed

In [ ]:
manifest = pd.read_csv('../models/studyA/manifest.csv')
controlManifest = manifest[manifest['sample_id'].str.contains('control')]
inulinManifest = manifest[manifest['sample_id'].str.contains('inulin')]

controlGrowth = micom.workflows.grow(controlManifest,
                              '../models/studyA/',
                              controlMedium, 
                              tradeoff = 0.7,
                              strategy = 'none',
                              threads = 10)
inulinGrowth = micom.workflows.grow(inulinManifest,
                              '../models/studyA/',
                              inulinMedium, 
                              tradeoff = 0.7,
                              strategy = 'none',
                              threads = 10)

## Compare SCFA Fluxes
We can now extract the production fluxes of SCFAs from each growth simulation, and construct a dataframe with both measured and predicted production rates.

In [ ]:
inulinProduction = micom.measures.production_rates(inulinGrowth)
controlProduction = micom.measures.production_rates(controlGrowth) # calculate predicted production rates
production = pd.concat([inulinProduction, controlProduction])

res = production[(production['name']==('butyrate'))|
                 (production['name']==('acetate'))|
                 (production['name']==('propionate'))] #isolate scfas

res = pd.pivot(res, index = 'sample_id',
                    columns = 'name',
                    values = 'flux') # pivot data 

res.rename(columns = {'acetate':'acetatePredicted',
                      'butyrate':'butyratePredicted',
                      'propionate':'propionatePredicted' #rename columns to identify predictions
                      }, inplace = True
          )
res.index = res.index.str.replace('PBS','control')


res['donor'] = res.index.str.split('_').str[0]
res['treatment'] = res.index.str.split('_').str[1]
res = res.groupby(['donor','treatment']).mean().reset_index()

res['sample_id'] = res['donor']+'_'+res['treatment']
res.set_index('sample_id', inplace = True)
res = pd.concat([res, scfa], axis = 1)
res = res.loc[:,~res.columns.duplicated()].copy()

res

## Scale by Biomass
To compare results between studies we will scale the predicted SCFAs by biomass. Without accurate qPCR, we will instead use metagenomic human reads as proxy. 

In [ ]:
abundance = pd.read_csv('../data/raw_data/studyA_original_abundance.csv', index_col = 0)

sampleReads = abundance.groupby('sample_id')['reads'].sum()
humanReads = abundance[abundance['g'] == 'Homo'].set_index('sample_id')['reads']
pctHuman = humanReads/sampleReads

pctHuman.index = pctHuman.index.str.rstrip('_0123456789')
pctHuman = pctHuman.groupby(level = 0).mean().to_dict()
res['pct_human'] = res.index.map(pctHuman)

res['acetatePredicted'] = res['acetatePredicted']*(1-res['pct_human'])
res['butyratePredicted'] = res['butyratePredicted']*(1-res['pct_human'])
res['propionatePredicted'] = res['propionatePredicted']*(1-res['pct_human'])

res

## Plot Results
Finally, we'll plot predicted vs measured fluxes against each other

In [ ]:
fig1 = (ggplot(
    res, aes(x = 'butyrateMeasured', y = 'butyratePredicted'))
    +geom_smooth(method = 'lm', linetype = '--')
    +geom_point(aes(color = 'treatment'), size = 8)
    +scale_color_manual(limits = ['control','inulin'], 
                        values = ['cornflowerblue', 'coral'],
                        labels = ['Control','Inulin'])    +labs(title='Butyrate',
          x='Measured Butyrate (mmol/L/h)',
          y = 'Predicted Butyrate (mmol/gDW/h)',
          color = 'Treatment')
    +theme(text = element_text(size=35, color = 'black'),panel_background=element_rect(fill = "white",
                                    colour = "white",size = 0.5, linetype = "solid"),panel_grid= element_blank(),
                                    axis_line = element_line(size = 2, linetype = "solid",colour = "black"),
                                    legend_position='right',axis_text_x=element_text(rotation = 20, hjust = 1))
)
fig1

In [ ]:
scipy.stats.linregress(res['butyratePredicted'], res['butyrateMeasured'])

In [ ]:
fig2 = (ggplot(
    res, aes(x = 'propionateMeasured', y = 'propionatePredicted'))
    +geom_smooth(method = 'lm', linetype = '--')
    +geom_point(aes(color = 'treatment'), size = 8)
    +scale_color_manual(limits = ['control','inulin'], 
                        values = ['cornflowerblue', 'coral'],
                        labels = ['Control','Inulin'])    +labs(title='Propionate',
          x='Measured Propionate (mmol/L/h)',
          y = 'Predicted Propionate (mmol/gDW/h)',
          color = 'Treatment')
    +theme(text = element_text(size=35, color = 'black'),panel_background=element_rect(fill = "white",
                                    colour = "white",size = 0.5, linetype = "solid"),panel_grid= element_blank(),
                                    axis_line = element_line(size = 2, linetype = "solid",colour = "black"),
                                    legend_position='right',axis_text_x=element_text(rotation = 20, hjust = 1))
)
fig2

In [ ]:
scipy.stats.linregress(res['propionatePredicted'], res['propionateMeasured'])

In [ ]:
res.to_csv('../results/studyA.csv')